In [1]:
from Bio import SeqIO
import pandas as pd
import pybedtools
import numpy as np
import gffutils

from IPython.core.display import HTML

In [151]:
annotated_exons = pybedtools.BedTool("/home/gpratt/clipper/clipper/data/regions/hg19_exons.bed").sort().saveas()
kd_exons = pybedtools.BedTool("/home/gpratt/projects/cryptic_exons/analysis/ad-hoc/SRR2002766_1.polyATrim.adapterTrim.rmRep_XS_tag.sorted.gtf")
ctrl_exons = pybedtools.BedTool("/home/gpratt/projects/cryptic_exons/analysis/ad-hoc/2002765/transcripts.gtf")

In [152]:
!gffutils-cli create /home/gpratt/projects/cryptic_exons/analysis/ad-hoc/SRR2002766_1.polyATrim.adapterTrim.rmRep_XS_tag.sorted.gtf --disable-infer

Traceback (most recent call last):
  File "/home/gpratt/anaconda/bin/gffutils-cli", line 5, in <module>
    pkg_resources.run_script('gffutils==0.8.2', 'gffutils-cli')
  File "/home/gpratt/anaconda/lib/python2.7/site-packages/distribute-0.6.35-py2.7.egg/pkg_resources.py", line 505, in run_script
    self.require(requires)[0].run_script(script_name, ns)
  File "/home/gpratt/anaconda/lib/python2.7/site-packages/distribute-0.6.35-py2.7.egg/pkg_resources.py", line 1245, in run_script
    execfile(script_filename, namespace, namespace)
  File "/home/gpratt/anaconda/lib/python2.7/site-packages/gffutils-0.8.2-py2.7.egg/EGG-INFO/scripts/gffutils-cli", line 297, in <module>
    rmdups
  File "/home/gpratt/anaconda/lib/python2.7/site-packages/argh-0.26.1-py2.7.egg/argh/dispatching.py", line 328, in dispatch_commands
    dispatch(parser, *args, **kwargs)
  File "/home/gpratt/anaconda/lib/python2.7/site-packages/argh-0.26.1-py2.7.egg/argh/dispatching.py", line 174, in dispatch
    for line in line

In [153]:
kd_db = gffutils.FeatureDB("/home/gpratt/projects/cryptic_exons/analysis/ad-hoc/SRR2002766_1.polyATrim.adapterTrim.rmRep_XS_tag.sorted.gtf.db")
ctrl_db = gffutils.FeatureDB("/home/gpratt/projects/cryptic_exons/analysis/ad-hoc/2002765/ctrl.db")

In [154]:
kd_exons = kd_exons.filter(lambda x: x[2] == "exon").sort().saveas()
ctrl_exons = ctrl_exons.filter(lambda x: x[2] == "exon").sort().saveas()

In [155]:
len(kd_exons), len(ctrl_exons)

(571868, 132896)

In [156]:
kd_annotated_exons = kd_exons.intersect(annotated_exons, s=True, u=True, f=1.0, r=True, sorted=True).saveas()
ctrl_annotated_exons = ctrl_exons.intersect(annotated_exons, s=True, u=True, f=1.0, r=True, sorted=True).saveas()

In [157]:
len(kd_annotated_exons), len(ctrl_annotated_exons)

(1767, 306)

Cufflinks is recovering a fairly large fraction of annotated exons, but there are look to be a pretty gigantic number of exons that are unannoated in both ctrl and kd, more in knockdown.  Which is useful.

In [158]:
kd_exon

In [159]:
kd_unannotated_exons = kd_exons.intersect(annotated_exons, s=True, f=1.0, r=True, sorted=True, v=True).saveas()
ctrl_unannotated_exons = ctrl_exons.intersect(annotated_exons, s=True, f=1.0, r=True, sorted=True, v=True).saveas()

In [164]:
foo = kd_exons.filter(lambda x: x.end == 111602981).saveas()

In [169]:
kd_unannotated_cryptic_exons = kd_exons.intersect(annotated_exons, s=True, v=True).intersect(hg19_genes, s=True).saveas()
ctrl_unannotated_cryptic_exons = ctrl_exons.intersect(annotated_exons, s=True, v=True).intersect(hg19_genes, s=True).saveas()

In [170]:
print len(kd_unannotated_exons), len(ctrl_unannotated_exons)
print len(kd_unannotated_cryptic_exons), len(ctrl_unannotated_cryptic_exons)

570101 132590
78 21


In [175]:
foo = """chr5:111602907-111602981
chr5:648215-648338
chr19:7169645-7169842
chr5:153416928-153417015
chr10:3141749-3142011
chr11:8797739-8801798
chr12:117226441-117226517
chr12:117227839-117228527
chr12:85689446-85689514
chr1:980268-980367
chr1:980272-980367
chr2:242608197-242608400
chr1:980268-980460
chr1:980272-980460
chr11:8801680-8801798
chr3:9510260-9510300
chr11:108368528-108368891
chr19:9112157-9112234
chr19:14560900-14561129
chr14:24630911-24631099
chr10:11968921-11971863
chr1:109438939-109439053
chr13:21374252-21374294
chr7:102227839-102228081
chr7:102128667-102128909
chr15:72557611-72557753
chr12:52631354-52631501
chr2:3462025-3462286
chr22:20110103-20110220
chr4:89319549-89319596
chr1:169337730-169337948
chr11:58384466-58384527
chr19:4492012-4492149
chr10:30731500-30731565
chr2:182775853-182775960
chr5:64836755-64836979
chr6:33626324-33626446
chr2:143722222-143728961
chr14:24634266-24634392
chrX:107465095-107465143
chr2:143728740-143728961""".split()

In [180]:
annotated_cryptic_exons = []
for line in foo:
    chrom, loc = line.split(":")
    start, stop = loc.split("-")
    annotated_cryptic_exons.append(pybedtools.create_interval_from_list([chrom, start, stop]))
    
annotated_cryptic_exons = pybedtools.BedTool(annotated_cryptic_exons)

In [183]:
len(annotated_cryptic_exons)

41

In [184]:
len(kd_unannotated_cryptic_exons)

78

In [187]:
annotated_cryptic_exons.intersect(kd_unannotated_cryptic_exons).head()

chr5	111602907	111602981
 chr10	3141749	3142011
 

In [173]:
kd_unannotated_cryptic_exons.head(n=100)

chr1	StringTie	exon	21080549	21081897	1000	-	.	gene_id "STRG.319"; transcript_id "STRG.319.4"; exon_number "1"; cov "38.064098";
 chr1	StringTie	exon	32605356	32605465	1000	+	.	gene_id "STRG.519"; transcript_id "STRG.519.2"; exon_number "2"; cov "24.732277";
 chr1	StringTie	exon	38477104	38477240	1000	+	.	gene_id "STRG.606"; transcript_id "STRG.606.1"; exon_number "2"; cov "1.569343";
 chr1	StringTie	exon	146649723	146650958	1000	-	.	gene_id "STRG.1384"; transcript_id "STRG.1384.1"; exon_number "1"; cov "149.560471";
 chr1	StringTie	exon	179825193	179825305	1000	-	.	gene_id "STRG.1877"; transcript_id "STRG.1877.3"; exon_number "5"; cov "65.685844";
 chr10	StringTie	exon	3141749	3142011	1000	+	.	gene_id "STRG.12684"; transcript_id "STRG.12684.2"; exon_number "4"; cov "24.359905";
 chr10	StringTie	exon	7452823	7452929	1000	-	.	gene_id "STRG.12729"; transcript_id "STRG.12729.1"; exon_number "21"; cov "8.296545";
 chr10	StringTie	exon	51338277	51338363	1000	-	.	gene_id "STRG.13002"; transc

Conclusions from today:
StringTie is fast as hell, but its max flow approach doesn't identify all possible exons.  This is probbably by design from the max flow algorithm.  

cufflinks looks like it idenitifes a bit more, I'm testing it out seriously right now.  We'll see how it goes, Going to put it on a 4 day time limit.  

In [171]:
print len(kd_unannotated_exons), len(ctrl_unannotated_exons)
print len(kd_unannotated_cryptic_exons), len(ctrl_unannotated_cryptic_exons)

570101 132590
78 21


In [140]:
kd_unannotated_cryptic_exons.saveas("cryptic_exons.gtf")

<BedTool(cryptic_exons.gtf)>

In [141]:
kd_unannotated_cryptic_exons.head(n=1000)

chr1	StringTie	exon	1779317	1783702	1000	-	.	gene_id "STRG.121"; transcript_id "STRG.121.3"; exon_number "12"; cov "0.750693";
 chr1	StringTie	exon	1784591	1784736	1000	-	.	gene_id "STRG.121"; transcript_id "STRG.121.4"; exon_number "12"; cov "14.770548";
 chr1	StringTie	exon	1811226	1811341	1000	-	.	gene_id "STRG.121"; transcript_id "STRG.121.2"; exon_number "12"; cov "0.719212";
 chr1	StringTie	exon	12227722	12227757	1000	+	.	gene_id "STRG.264"; transcript_id "STRG.264.1"; exon_number "2"; cov "1.383438";
 chr1	StringTie	exon	55680279	55680370	1000	-	.	gene_id "STRG.935"; transcript_id "STRG.935.1"; exon_number "68"; cov "2.165092";
 chr1	StringTie	exon	161171100	161171123	1000	+	.	gene_id "STRG.1803"; transcript_id "STRG.1803.1"; exon_number "1"; cov "1.000000";
 chr1	StringTie	exon	212977662	212977662	1000	+	.	gene_id "STRG.2247"; transcript_id "STRG.2247.14"; exon_number "6"; reference_id "ENST00000497768.2"; ref_gene_id "ENSG00000203705.5"; ref_gene_name "TATDN3"; cov "0.000000";

In [68]:
print 83806 + 47220,
print 32267 + 19179 
#unannotated + annotated is equal to total.  Good sanity check.  

131026 51446


In [69]:
overlapping_annotated_kd_ctrl = kd_unannotated_exons.intersect(ctrl_unannotated_exons, s=True, u=True, f=1.0, r=True, sorted=True).saveas()
overlapping_annotated_ctrl_kd = ctrl_unannotated_exons.intersect(kd_unannotated_exons, s=True, u=True, f=1.0, r=True, sorted=True).saveas()


In [70]:
len(overlapping_annotated_kd_ctrl), len(overlapping_annotated_ctrl_kd)

(1661, 1676)

This is interesting, cryptic exons don't really overlap, but there are bunches of cryptic exons in both ctrl and kd that weren't reported.  Time to dig into this.  

Important Questions:

1. Do these unannoated exons have correct splice junctions?
2. What is the PSI score / general coverage of these exons?

#1. What do splice junctions look like?

In [90]:
def make_interval(interval):
    #Might still need this code
    if interval.start > 2:
        interval.start -= 2
        interval.stop += 2
    transcript_gene_id = (interval.attrs['transcript_id'][0], [0])
    if interval.start > interval.stop:
        print interval
    name = "{}:{}-{}({}) {} {} {}".format(interval.chrom, 
                                          interval.start, 
                                          interval.stop, 
                                          interval.strand,
                                          interval.attrs['exon_number'], 
                                          interval.attrs['transcript_id'],
                                          interval.attrs['gene_id'])
    interval = pybedtools.create_interval_from_list([interval.chrom, interval.start, interval.stop, name, "0", interval.strand])
    return interval 

In [72]:
ctrl_annotated_exons_bed = ctrl_annotated_exons.each(make_interval).saveas()
ctrl_annotated_exons_fasta = ctrl_annotated_exons_bed.sequence(fi="/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa", fo="ctrl_annotated_exons.fasta", name=True, fullHeader=True, s=True)

In [73]:
def make_exon_count_dict(db):
    exon_count_dict = {}
    for transcript in db.features_of_type("transcript"):
        exons = list(db.children(transcript, featuretype="exon", order_by="start"))

        exon_numbers = [interval.attributes['exon_number'][0] for interval in exons]
        transcript_id = exons[0].attributes['transcript_id'][0]
        transcript_gene_id = (transcript.attributes['transcript_id'][0], transcript.attributes['gene_id'][0])

        exon_count_dict[transcript_gene_id] = {"min": min(exon_numbers),
                                 "max": max(exon_numbers)}

        if transcript_id in exon_count_dict:
            print transcript_id
    return exon_count_dict
    
ctrl_exon_count_dict = make_exon_count_dict(ctrl_db)
kd_exon_count_dict = make_exon_count_dict(kd_db)

In [74]:
def count_splice_junctions(fasta_file, exon_count_dict):
    result = {}
    for exon in SeqIO.parse(fasta_file, "fasta"):
        strand = exon.name[-2]
        five_prime = exon.seq[-2:].upper() == "GT"
        three_prime = exon.seq[:2].upper() == "AG"
        exon_number = int(exon.description.split()[1])
        transcript_id =  exon.description.split()[2]
        gene_id =  exon.description.split()[3]
        try:
            max_exon = exon_count_dict[(transcript_id, gene_id)]['max']
            min_exon = exon_count_dict[(transcript_id, gene_id)]['min']
        except KeyError:
            continue

        if strand == "+":
            if exon_number == int(min_exon):
                five_prime = "first_exon"
            if exon_number == int(max_exon):
                three_prime = "last_exon"
        elif strand == "-":
            if exon_number == int(max_exon):
                five_prime = "first_exon"
            if exon_number == int(min_exon):
                three_prime = "last_exon"
        else:
            print "strand not right"
            continue
            
            #raise("Strand not right")
        result[exon.id] = {"exon_number": exon_number,
                           "five_prime": five_prime, 
                           "three_prime": three_prime,
                           "five_prime_seq": "".join(exon.seq[-2:]).upper(), 
                           "three_prime_seq": "".join(exon.seq[:2]).upper()}

    result = pd.DataFrame(result).T
    return result

#Conclusions from initial QC
1. I'm missing some transcripts using my approach of getting an exon counting dict
2. There are some non-canonical splice events, about 1/4 of my data doesn't have canonical splice sites, breif manual exmaination didn't show anything suprising about these splice sites.  Above is the frequencies of the non-canonoical splice sites, there isn't anything odd about the distribution.  One thing I noticed is that we are likely to have one good 3' or 5' site and one bad 3' or 5' site on the exon level.  This is the weakness of taking an exon centric approach instead of an intron centric approach.  

I think I'm good to move on to looking at differences between cananoical exons and non-canonical exons.  Going to try that now

#Non-canonical exon Stats

In [91]:
ctrl_annotated_exons_bed = ctrl_annotated_exons.each(make_interval).saveas()
ctrl_annotated_exons_fasta = ctrl_annotated_exons_bed.sequence(fi="/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa", fo="ctrl_annotated_exons.fasta", name=True, fullHeader=True, s=True)

kd_annotated_exons_bed = kd_annotated_exons.each(make_interval).saveas()
kd_annotated_exons_fasta = kd_annotated_exons_bed.sequence(fi="/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa", fo="kd_annotated_exons.fasta", name=True, fullHeader=True, s=True)

ctrl_unannotated_exons_bed = ctrl_unannotated_exons.each(make_interval).saveas()
ctrl_unannotated_exons_fasta = ctrl_unannotated_exons_bed.sequence(fi="/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa", fo="ctrl_unannotated_exons.fasta", name=True, fullHeader=True, s=True)

kd_unannotated_exons_bed = kd_unannotated_exons.each(make_interval).saveas()
kd_unannotated_exons_fasta = kd_unannotated_exons_bed.sequence(fi="/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa", fo="kd_unannotated_exons.fasta", name=True, fullHeader=True, s=True)

WARNING. chromosome (ERCC-00002) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00004) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00009) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00012) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00019) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00024) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00025) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00033) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00039) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00042) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00043) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00044) was not found in the FASTA file. Skipping.
WARNING. chromosome (ERCC-00046) was not found in the FASTA file. Skipping.
WARNING. chr

In [92]:
ctrl_annotated_junctions = count_splice_junctions("ctrl_annotated_exons.fasta", ctrl_exon_count_dict)
kd_annotated_junctions = count_splice_junctions("kd_annotated_exons.fasta", kd_exon_count_dict)
ctrl_unannotated_junctions = count_splice_junctions("ctrl_unannotated_exons.fasta", ctrl_exon_count_dict)
kd_unannotated_junctions = count_splice_junctions("kd_unannotated_exons.fasta", kd_exon_count_dict)

In [93]:
ctrl_annotated_junctions.groupby("five_prime").count()

,exon_number,five_prime_seq,three_prime,three_prime_seq
five_prime,,,,
False,4650,4650,4650,4650
True,16663,16663,16663,16663
first_exon,7056,7056,7056,7056


In [94]:
kd_annotated_junctions.groupby("five_prime").count()

,exon_number,five_prime_seq,three_prime,three_prime_seq
five_prime,,,,
False,14383,14383,14383,14383
True,70637,70637,70637,70637
first_exon,21306,21306,21306,21306


In [98]:
kd_unannotated_junctions

,exon_number,five_prime,five_prime_seq,three_prime,three_prime_seq
chr10:100003845-100004323(+),9,False,AA,last_exon,AG
chr10:100003845-100004656(+),10,False,TC,True,AG
chr10:100007444-100028009(-),1,first_exon,GA,last_exon,GG
chr10:100143319-100174943(-),1,first_exon,TT,last_exon,AG
chr10:100154972-100155066(-),1,first_exon,GT,last_exon,CT
chr10:100175952-100206686(-),1,first_exon,TC,last_exon,GC
chr10:100177929-100178016(-),2,True,GT,True,AG
chr10:100179799-100180128(-),2,first_exon,GT,False,TT
chr10:100183362-100183427(-),1,True,GT,last_exon,AG
chr10:100183634-100183646(-),1,False,AG,last_exon,AG


In [95]:
ctrl_unannotated_junctions.groupby("five_prime").count()`

,exon_number,five_prime_seq,three_prime,three_prime_seq
five_prime,,,,
False,8496,8496,8496,8496
True,7202,7202,7202,7202
first_exon,12108,12108,12108,12108


In [96]:
kd_unannotated_junctions.groupby("five_prime").count()

,exon_number,five_prime_seq,three_prime,three_prime_seq
five_prime,,,,
False,35598,35598,35598,35598
True,34082,34082,34082,34082
first_exon,56106,56106,56106,56106


A breif look at these exons doesn't make any sense.  I'm trying to get cufflinks running faster, but until its totally done, I don't think I can use this approach.  I'll switch to trying to find novel exons via STAR now.

#Junction Based Approach for annotating exons

I'm going to try something hacky to identify unannotated exons in rna-seq data

1. Create coverage map of all of hg19 using HTSeq and or a wiggle file (I'll try with HTSeq first)
2. Load in all splice junction annotations from STAR
3. Start with the first splice junction, create a splice graph based on coverage to define exons and any splice junctions contained in that covered region.  Any region and has a two splice junctions can be punatively defined as an exon.  I'm NOT going to try to infer full transcripts, just exons in those transcripts

In [ ]:
import HTSeq

In [491]:
bam = HTSeq.BAM_Reader("/home/gpratt/projects/cryptic_exons/analysis/ling_et_al_v1/SRR2002765_1.polyATrim.adapterTrim.rmRep.sorted.rg.bam")

In [496]:
coverage = HTSeq.GenomicArray( "auto", stranded=True, typecode="i" )

In [499]:
HTSeq.pair_SAM_alignments??

In [504]:
def get_bam_coverage(bamfile):
    """
    
    Given a bam file returns a properly covered htseq coverage file (this is slow)
    
    """
    bam = HTSeq.BAM_Reader(bamfile)
    coverage = HTSeq.GenomicArray("auto", typecode="i", stranded=True)
    for read in bam:
        if read.aligned:
            for cigop in read.cigar:
                if cigop.type != "M":
                    continue
                coverage[cigop.ref_iv] += 1
    return coverage

In [ ]:
coverage = get_bam_coverage("/home/gpratt/projects/cryptic_exons/analysis/ling_et_al_v1/SRR2002765_1.polyATrim.adapterTrim.rmRep.sorted.rg.bam")

In [503]:
for cigop in r1.cigar:

[< CigarOperation: 100 base(s) matched on ref iv chr1:[11389,11489)/+, query iv [0,100) >]

In [494]:
print r1

<SAM_Alignment object: Paired-end read 'SRR2002765.64014174' aligned to chr1:[11389,11489)/+>


In [495]:
print r2

<SAM_Alignment object: Paired-end read 'SRR2002765.64014174' aligned to chr1:[11524,11624)/->


#Intron Centric Approach

In [ ]:
for transcript in ctrl_db.features_of_type("transcript"):
    exons = ctrl_db.children(transcript, featuretype="exon", order_by="start")
    result = list(ctrl_db.interfeatures(exons))
    if len(result) > 5:
        break

def _to_bed(interval):
    """
    interval: gffutils interval

    converts gffutils to bed format, setting id to be the 
    id of the gene

    """
    gene_id = interval.attributes['gene_id']
    if type(gene_id) is list:
        gene_id = gene_id[0]        
    return pybedtools.create_interval_from_list([interval.chrom, interval.start, 
            interval.stop, gene_id, "0", interval.strand])

In [ ]:
for x in ctrl_db.features_of_type("transcript"):
    print x
    break

In [ ]:
list(ctrl_db.children(transcript_id, featuretype="transcript_id"))